# Простейшая нейронная сеть

Пример инспирирован: https://github.com/stmorgan/pythonNNexample

Видео: https://youtu.be/h3l4qz76JhQ.

## Функция активации и её производная

Производная для градиентного спуска

In [1]:
import numpy as np
from numpy import array as na

In [2]:
# Функции numpy и её операции покомпонентно работают с векторами

def σ(x):
    return 1 / (1 + np.exp(-x))

def dσdx(x):
    return σ(x)*(1-σ(x))

λ = 0.15  # тормозилка
iterations = 40000  # зубрилка

Обучающие данные. Во входном слое также есть т.н. *нейрон смещения*, который всегда 1

In [3]:
# Входные данные — набор векторов-столбцов
X = [
    na([[0],[0],[1]]),
    na([[0],[1],[1]]),
    na([[1],[0],[1]]),
    na([[1],[1],[1]])
]

# Выходные данные — XOR и `=>` — тоже набор векторов-столбцов
Z = [
    na([[0], [1]]),
    na([[1], [1]]),
    na([[1], [0]]),
    na([[0], [1]])
]

Инициализируем линейные операторы, имитирующие синапсы, случайными значениями. 

In [4]:
np.random.seed(2)

# Инициализируем матрицы весов (соответствуют синапсам нейронной сети)
W_0 = 2 * np.random.random((4,3)) - 1  # Матрица весов 4x3 — 4 нейрона в скрытом слое из 3 входных нейронов
W_1 = 2 * np.random.random((2,4)) - 1  # Матрица весов 2x4 — 2 выходных нейрона из 4 скрытых

Модель сети:

<img src="img/neural1.png" style="width: 50%;" />

Обучение с обратным распространением ошибки:

<img src="img/backpro1.png" style="width: 50%;" />


Запустим в цикле обучение, повторяя одни и те же обучающие примеры 

In [5]:
for j in range(iterations):
    for x, z in zip(X, Z):
    
        # Применим нейронную сеть
        l_0 = x
        l_1 = σ(W_0 @ l_0)
        l_2 = σ(W_1 @ l_1)
        
        # Обратное распространение ошибки
        
        # ---- Выходной слой ----
        
        ε_l_2 = z - l_2                 # ε_l_2 — ошибка (со знаком минус) на выходе —
                                        # сколько надо минус сколько получилось.
        
        Δ_i_2 = λ * ε_l_2 * dσdx(l_2)   # Как должны были измениться входы нейронов выходного слоя
                                        # градиентный спуск в чистом виде. Несмотря на то, что у нас dσ/dx,
                                        # фактически мы используем её, как производную по линейным операторам
                                        # нейронной сети, а x у нас на каждой итерации фиксирован.

        # ---- Скрытый слой ----
            
        ε_l_1 = W_1.T @ Δ_i_2           # Неочевидное место (1) — как мы считаем ошибку (со знаком минус)
                                        # в скрытом слое.

        Δ_i_1 = λ * ε_l_1 * dσdx(l_1)   # Как должны были измениться входы нейронов скрытого слоя
                                        # опять градиентный спуск в чистом виде.
    
        # ---- Коррекция весов ----
        
        W_1 += Δ_i_2 @ l_1.T            # Неочевидное место (2) — 
        W_0 += Δ_i_1 @ l_0.T            # Как мы корректируем операторы
    
    if j % 10000 == 0:   # Раз в 1000 итераций печатаем среднее значение квадрата отклонения 
        print(f"Error: {np.square(ε_l_2).mean()}")

Error: 0.23139668302794075
Error: 0.0005610036910077208
Error: 8.87985676657125e-05
Error: 4.701233539415214e-05


### Неочевидные места (линейные индексы пишем наверху)

Связаны с тем, что мы пытаемся пройти обратно не «через обратные функции и обратные матрицы», а «через производные функций и линейных операторов». Потому что прямо идёт сигнал, а обратно идёт ошибка, и мы хотим смоделировать ситуацию, при которой изменения в параметрах будут вызывать нужные изменения в ошибке.

1. $\varepsilon l_1 = W_1^T \times \Delta i_2$. Мы хотим выяснить, насколько ошибся слой $l_1$ (ошибся в том, что вызвало ошибку в следующем слое =)).
    * $i_2^i = \sum_j W_1^{i,j} l_1^j$.
    * Мы видим, что приращение $\varepsilon l_1^{j}$, приводит к изменениям $\Delta i_2^i = W_1^{i,j} l_1^{j}$. И поэтому мы в $l_1^j$ «отправляем» назад по тем же связям получившуюся в $i_2$ ошибку в виде $\sum_i W_1^{i,j} \Delta i_2^i$.
2. `W_1 += Δ_i_2 @ l_1.T`. Примерно та же идея.
    * $W_1$ — насколько сильно выходы $l_1$ действуют на входы $i_2$, а именно $i_2^i = \sum_j w_1^{i,j} l_1^j$.
    * Соответственно, если кто-то $j$-й подействовал ($l_1^j$) неправильно на $i$-го ($\Delta_{i_2}^i$), мы это учтём пропорциолнально тому, какую ошибку это вызвало, и насколько он сам был возбуждён.
    * И скорректируем их связи.

А теперь давайте протестируем сеть...

In [6]:
def predict(arg):
    arg += [1]
    l_1 = σ(W_0 @ arg)
    l_2 = σ(W_1 @ l_1)
    return l_2

print(predict([0,0]))
print(predict([0,1]))
print(predict([1,0]))
print(predict([1,1]))

[0.00958842 0.99867086]
[0.98627491 0.99999998]
[0.9917872  0.00367186]
[0.00888672 0.99826763]
